<a href="https://colab.research.google.com/github/jamiemcgarry/Using_Frame_Semantic_Transfomer_on_historical_texts/blob/main/Euclidean_distance_measurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I show demonstrate how I conducted the euclidean distance test based on the significant semantic frames between all four auhtors. I will demonstrate with Burke and Paine.

In [1]:
!pip install spacy

In [2]:
!pip install nltk

In [3]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [5]:
import nltk
nltk.download('framenet_v17')

[nltk_data] Downloading package framenet_v17 to /root/nltk_data...
[nltk_data]   Unzipping corpora/framenet_v17.zip.


True

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd

file_path = '/content/drive/My Drive/Burke_Paine_significance_f.csv'

Burke_Paine_significance_df = pd.read_csv(file_path)

In the two cells above replace these with code to import from wherever you have saved 'Burke_Paine_significance_f.csv'

In [14]:
Burke_Paine_significance_df.set_index(Burke_Paine_significance_df.columns[0], inplace=True)

Burke_Paine_significance_df.index.name = 'Frames'

In [19]:
Burke_Paine_frames = Burke_Paine_significance_df.index.tolist()

In [16]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [21]:
from nltk.corpus import framenet as fn
import re
words = {}

for frame in fn.frames():
    frame_name = frame.name
    if frame_name in Burke_Paine_frames:
      array = []
      for lex_unit in frame.lexUnit.values():
        array.append(lex_unit.name)
      lexias = r"\n".join(array)
      lexias = re.sub(r'\.\w+\\n', ' ', lexias)
      lexias = re.sub(r'\.a', '', lexias)
      lexias = re.sub(r'\.n', '', lexias)
      lexias = re.sub(r'\.s', '', lexias)
      lexias = re.sub(r'\.v', '', lexias)
      lexias = re.sub(r'[()\[\]]', '', lexias)
      tokenized = word_tokenize(lexias)
      if len(tokenized) == 10:
        ex = nlp(lexias)
        final_lexias = ex.vector
        words[frame_name] = final_lexias
      elif len(tokenized) > 10:
        l_tokenized = tokenized[:10]
        ex = nlp(" ".join(l_tokenized))
        final_lexias = ex.vector
        words[frame_name] = final_lexias
      else:
        a = [lexias, frame_name]
        a_1 = re.sub(r'\_', ' ', " ".join(a))
        new_tokenized = word_tokenize(a_1)
        if len(new_tokenized) == 10:
           ex = nlp(a_1)
           final_lexias = ex.vector
           words[frame_name] = final_lexias
        elif len(new_tokenized) < 10:
          b = [lexias, frame_name] + [FE for FE in frame.FE.keys()]
          b_1 = re.sub(r'\_', ' ', " ".join(b))
          new_tokenized = word_tokenize(b_1)
          if len(new_tokenized) == 10:
            ex = nlp(b_1)
            final_lexias = ex.vector
            words[frame_name] = final_lexias
          elif len(new_tokenized) > 10:
            c = new_tokenized[:10]
            c_1 = " ".join(c)
            ex = nlp(c_1)
            final_lexias = ex.vector
            words[frame_name] = final_lexias
          else:
            continue

    else:
      continue

#words

The point of this cell is to create a dictionary of the frame name in the keys() and spacy's 300 dimensional vectors that represent that frame in the value().To do this I take the semantic frames directly from FrameNet and if the frame happens to be one of the significant semantic frames from Burke and Paine comparison, then the lexical units of that frame are extracted. This code makes sure that each list if lexical units is exactly 10 words in length,  the spacy 300 dimensional vectors consistent.

In [23]:
import pandas as pd

vector_dataframe = pd.DataFrame(index=words.keys(), data=words.values())
vector_dataframe['author'] = ''

for item in vector_dataframe.index:
  if PB_significane_df.loc[item, 'Burke'] > PB_significane_df.loc[item, 'Paine']:
    vector_dataframe.loc[item, 'author'] = 'Burke'
  else:
    vector_dataframe.loc[item, 'author'] = 'Paine'


#vector_dataframe

In [34]:
Burke_df = vector_dataframe[vector_dataframe['author'] == 'Burke']
Burke_df = Burke_df.drop(columns = ['author'])
Paine_df = vector_dataframe[vector_dataframe['author'] == 'Paine']
Paine_df = Paine_df.drop(columns = ['author'])
#Paine_df
#Burke_df

I then put that dictionary into a dataframe and split the dataframe into two. One being the vectors for Burke's unique semantic frames and another for Paines

In [35]:
centroid_Burke = Burke_df.mean(axis=0)


In [36]:
centroid_Paine = Paine_df.mean(axis=0)


I then take the mean, or otherwise known as centroid, for each dimension in the data frame. This gives 300 dimensional vectors for the author as a whole.

In [37]:
from scipy.spatial.distance import euclidean

distance = euclidean(centroid_Paine, centroid_Burke)

print(f"the semantic distance between Burke and Paine measured by the euclidean distance is: {distance}")

the semantic distance between Burke and Paine measured by the euclidean distance is: 7.316391468048096


I then conducted the eculidean distance measurement